<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [ ]:
!pip install category_encoders

In [107]:
# imports
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adamax

In [ ]:
seed = 7
np.random.seed(seed)

In [16]:
# loading the data
FILEPATH = 'WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
df = pd.read_csv(FILEPATH)

In [17]:
df.head() # dataframe before it has been cleaned

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# cleaning data

# taking out customer_ID and saving it in its own df
customer_id_df = df['customerID']
df = df.drop('customerID', axis=1)

# transforming binary columns
columns = df.columns
map2binary = {'Female': 0,
              'Male': 1,
              'No': 0,
              'Yes': 1,
              0:0,
              1:1,
              'No phone service': 0,
              'No internet service': 0}

for each in columns:
    if df[each].value_counts().shape[0] == 2:
        df[each] = df[each].map(map2binary)

df['MultipleLines'] = df['MultipleLines'].map(map2binary)
df['OnlineSecurity'] = df['OnlineSecurity'].map(map2binary)
df['OnlineBackup'] = df['OnlineBackup'].map(map2binary)
df['DeviceProtection'] = df['DeviceProtection'].map(map2binary)
df['TechSupport'] = df['TechSupport'].map(map2binary)
df['StreamingTV'] = df['StreamingTV'].map(map2binary)
df['StreamingMovies'] = df['StreamingMovies'].map(map2binary)

# one hot encoding
columns_to_onehot = ['InternetService', 'Contract', 'PaymentMethod']
encoder = ce.OneHotEncoder(use_cat_names=True, cols=columns_to_onehot)
df = encoder.fit_transform(df)

# changing TotalChanges from a string
df['TotalCharges'] = df['TotalCharges'].apply(lambda x: x.strip())
for i in range(len(df['TotalCharges'])):
    if df['TotalCharges'][i] == '':
        df['TotalCharges'][i] = df['tenure'][i] * df['MonthlyCharges'][i]
df['TotalCharges'] = df['TotalCharges'].apply(lambda x: float(x))

# scaling the data
scaler = StandardScaler()
columns_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [73]:
df.head() # dataframe after it has been cleaned

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,-1.277445,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,-1.160323,-0.992611,0
1,1,0,0,0,0.066327,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,-0.259629,-0.172165,0
2,1,0,0,0,-1.236724,1,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,-0.362660,-0.958066,1
3,1,0,0,0,0.514251,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,-0.746535,-0.193672,0
4,0,0,0,0,-1.236724,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0.197365,-0.938874,1


In [ ]:
# baseline keras model
train, test = train_test_split(df, test_size=0.33, random_state=42)

x_cols = train.columns[:-1]
y_col = train.columns[-1]

X_train = train[x_cols]
X_test = test[x_cols]
y_train = train[y_col]
y_test = test[y_col]

inputs = X_train.shape[1]
epochs = 10
batch_size = 10

model = Sequential()
model.add(Dense(27, activation='relu', input_shape=(inputs,)))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

In [123]:
# hyperparamater tuning
X = df[x_cols]
Y = df[y_col]

def create_model(lr, optimizer='adam', act1='relu', act2='sigmoid',
                 dropout_rate=0.0, init_mode='uniform', neurons=1):
    adamax = Adamax(learning_rate=lr)
    model = Sequential()
    model.add(Dense(neurons, activation=act1, kernel_initializer=init_mode, input_shape=(inputs,)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation=act2, kernel_initializer=init_mode))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10], # out of [10, 20, 40, 60, 80, 100] 10 was the best
              'epochs': [80], # out of [1, 10, 50, 60, 70, 80, 90, 100, 150, 200] 80 was the best
              'optimizer': ['Adamax'], # out of ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'] Adamax was the best
              'lr': [.1], # out of [.001, .01, .1, .2, .3, .5] .1 was the best
              #'momentum': , this isnt used in the adamax optimizer
              'act1': ['tanh'], # out of['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'] tanh was the best
              'act2': ['sigmoid'], # out of ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'] sigmoid was the best
              'init_mode': ['glorot_normal'], # out of ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'] glorot_normal was the best
              'dropout_rate': [0.1], # out of [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 0.1 was the best
              'neurons': [50] # out of [1, 5, 10, 15, 20, 25, 30, 50, 100] 50 was the best
}
              
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8057650923728943 using {'act1': 'tanh', 'act2': 'sigmoid', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 80, 'init_mode': 'glorot_normal', 'lr': 0.1, 'neurons': 50, 'optimizer': 'Adamax'}
Means: 0.7975282669067383, Stdev: 0.008164894290084785 with: {'act1': 'tanh', 'act2': 'sigmoid', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 1, 'init_mode': 'glorot_normal', 'lr': 0.1, 'neurons': 50, 'optimizer': 'Adamax'}
Means: 0.8017894268035889, Stdev: 0.006528878007612706 with: {'act1': 'tanh', 'act2': 'sigmoid', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 10, 'init_mode': 'glorot_normal', 'lr': 0.1, 'neurons': 50, 'optimizer': 'Adamax'}
Means: 0.8040607452392579, Stdev: 0.005836316571979365 with: {'act1': 'tanh', 'act2': 'sigmoid', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 50, 'init_mode': 'glorot_normal', 'lr': 0.1, 'neurons': 50, 'optimizer': 'Adamax'}
Means: 0.8030675530433655, Stdev: 0.005004676230014587 with: {'act1': 'tanh', 'act2': 'sigmoid', 'batch_size': 10, 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?